In [1]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer
import torch
from transformers import AutoModelForTokenClassification, Trainer, TrainingArguments
from datasets import load_metric


/home/zjing2/capstone_venv/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Process Data

## Load Data

In [2]:
def create_dataset(file):

    with open(file, 'r') as f:
        lines = f.read()
    
    sentences = lines.split('\n\n')
    all_sent = []
    all_tag = []
    
    for sent in sentences:
        rows = sent.split("\n")
        words = []
        tags = []
        
        for r in rows[:-1]:
            w, t = r.split(" ")
            words.append(w)
            tags.append(t)
    
        all_sent.append(words[:])
        all_tag.append(tags[:])

    return pd.DataFrame({"tokens":all_sent, "ner_tags":all_tag})
    

In [3]:
train_df = None

for i in [1,2,4,5]:
    file = "acl-out{}.conll".format(i)
    if train_df is None:
        train_df = create_dataset(file)
    else:
        train_df = pd.concat([train_df, create_dataset(file)], ignore_index=True)

val_df = create_dataset("acl-out3.conll")



In [4]:
test_data = pd.read_csv('test_private.csv', index_col="id", skip_blank_lines=False)
inputs = test_data["input"].to_list()

for i in range(len(inputs)):
    if pd.isna(inputs[i]):
        inputs[i] = "END"

test_data["input"] = inputs
test_data['target'] = "O"

In [5]:
test_texts = []
test_tags = []
test_words = test_data['input'].to_list()

sentence = []
for word in test_words[1:]:
    if word == "END":
        sentence.append(word)
        test_texts.append(sentence[:])
        test_tags.append(["O"]*len(sentence))
        sentence = []
    else:
        sentence.append(word)
sentence.append("END")

test_texts.append(sentence[:])
test_tags.append(["O"]*len(sentence))

test_df = pd.DataFrame({"tokens":test_texts, "ner_tags":test_tags})

## Tokenize Data

In [6]:
tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")

labels = set()
for i, r in train_df.iterrows():    
    labels = labels.union(set(r['ner_tags']))

tag2id = {tag: id for id, tag in enumerate(labels)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [7]:
train_texts = train_df['tokens'].to_list()
val_texts = val_df['tokens'].to_list()
test_texts = test_df['tokens'].to_list()

train_tags = train_df['ner_tags'].to_list()
val_tags = val_df['ner_tags'].to_list()
test_tags = test_df['ner_tags'].to_list()

train_encodings = tokenizer(train_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True, max_length=512)
val_encodings = tokenizer(val_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True, max_length=512)
test_encodings = tokenizer(test_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True, max_length=512)


In [8]:
def encode_tags(tags, encodings, test=False):
    labels = [[tag2id[tag] for tag in doc] for doc in tags]
    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):

        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
        arr_offset = np.array(doc_offset)

        if test:
            doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = tag2id["O"]
        else:
            doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels

        encoded_labels.append(doc_enc_labels.tolist())

    return encoded_labels

train_labels = encode_tags(train_tags, train_encodings)
val_labels = encode_tags(val_tags, val_encodings)
test_labels = encode_tags(test_tags, test_encodings, True)


In [9]:
class PaperDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_encodings.pop("offset_mapping") # we don't want to pass this to the model
val_encodings.pop("offset_mapping")
test_encodings.pop("offset_mapping")
train_dataset = PaperDataset(train_encodings, train_labels)
val_dataset = PaperDataset(val_encodings, val_labels)
test_dataset = PaperDataset(test_encodings, test_labels)


# Fine-tuning with trainer

## Setting Training Arguments

In [10]:

training_args = TrainingArguments(
    output_dir='./results',          
    num_train_epochs=10,              # total number of training epochs
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    evaluation_strategy = "epoch",
    # learning_rate=2e-5,
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

model = AutoModelForTokenClassification.from_pretrained("allenai/scibert_scivocab_uncased", num_labels=len(labels))


Some weights of BertForTokenClassification were not initialized from the model checkpoint at allenai/scibert_scivocab_uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Setting Metrics

In [11]:
metric = load_metric("seqeval")
metric.compute(predictions=[labels], references=[labels])
label_list = list(labels)

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }


/tmp/ipykernel_1416806/848863643.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


## Setting Trainer

In [12]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    # data_collator=data_collator,
    eval_dataset=val_dataset,             # evaluation dataset
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,

)

## Train

In [13]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,2.440100,1.833861,0.000000,0.000000,0.000000,0.784964
2,0.711400,0.680808,0.000000,0.000000,0.000000,0.879547
3,0.506600,0.595269,0.500000,0.020000,0.038462,0.881164
4,0.345100,0.483594,0.425532,0.200000,0.272109,0.888440
5,0.192700,0.360613,0.533333,0.240000,0.331034,0.898949
6,0.132100,0.297747,0.652174,0.450000,0.532544,0.920776
7,0.077100,0.329308,0.550725,0.380000,0.449704,0.911884
8,0.049600,0.339178,0.690141,0.490000,0.573099,0.927243
9,0.035600,0.348912,0.626667,0.470000,0.537143,0.924818
10,0.024300,0.356296,0.666667,0.580000,0.620321,0.924010


/home/zjing2/capstone_venv/lib64/python3.9/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zjing2/capstone_venv/lib64/python3.9/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=260, training_loss=0.49375794031299075, metrics={'train_runtime': 629.6484, 'train_samples_per_second': 6.448, 'train_steps_per_second': 0.413, 'total_flos': 408232295062200.0, 'train_loss': 0.49375794031299075, 'epoch': 10.0})

## Eval

In [14]:
trainer.evaluate()

{'eval_loss': 0.35629627108573914,
 'eval_precision': 0.6666666666666666,
 'eval_recall': 0.58,
 'eval_f1': 0.6203208556149732,
 'eval_accuracy': 0.9240097008892482,
 'eval_runtime': 0.9665,
 'eval_samples_per_second': 38.282,
 'eval_steps_per_second': 1.035,
 'epoch': 10.0}

## Test

In [22]:
predictions, labels, _ = trainer.predict(test_dataset)
predictions = np.argmax(predictions, axis=2)


In [18]:
predict = []
groundtruth = []
for prediction, label in zip(predictions, labels):
    level_p = []
    level_l = []
    for p, l in zip(prediction, label):
        # print(p, l)
        if l != -100 and p != -100:
            # print(p, id2tag[p], l, id2tag[l])
            level_p.append(id2tag[p])
            level_l.append(id2tag[l])
    predict.append(level_p)
    groundtruth.append(level_l)

In [20]:
# Set DocString
res = ["O"]
# Add "O" to sentence that exceed MAX_LENGTH of the pre-trained model, which is 512 tokens.
predict[676] += ["O"]*57
predict[466] += ["O"]*3

for i in range(len(predict)):
    res += predict[i][:]

# Remove the last END
res.pop()


'O'

## Output Result

In [21]:
test_data['target'] = res
# test_data.drop(columns=['input']).to_csv("test_res.csv")
# test_data.to_csv("test_res_with_input.csv")